In [1]:
import argparse
import os
import random
import shutil
import time
import warnings
import pickle
import numpy as np
import math
import sys
import copy
import re
import pandas as pd
import matplotlib.pyplot as plt
import json
import cv2
from itertools import compress

import torch
import torch.nn as nn
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer,HookBase
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer,ColorMode,GenericMask
from detectron2.structures import BoxMode
from detectron2.evaluation import COCOEvaluator,inference_on_dataset
from detectron2.data import build_detection_test_loader,DatasetMapper,build_detection_train_loader,MetadataCatalog,DatasetCatalog
import detectron2.data.transforms as T
import detectron2.utils.comm as comm

import ray
import time

import uuid as uuid
from operator import itemgetter
import seaborn as sns

import shapely
import shapely.geometry
from shapely.geometry import Polygon,MultiPolygon,GeometryCollection
from shapely.validation import make_valid
from shapely.geometry import mapping
#import geopandas as gpd

#import imgfileutils as imf
#import segmentation_tools as sgt
from aicsimageio import AICSImage, imread
from skimage import measure, segmentation
from skimage.measure import regionprops
from skimage.color import label2rgb
#import progressbar
from IPython.display import display, HTML
#from MightyMosaic import MightyMosaic

import glob
from PIL import Image
import csv

In [112]:
# setup directory
root = r'/Users/lovely_shufan/'

project_dir = root + r'Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2022/0_inference_using_MaskRCNN_2021/'
output_dir = project_dir + r'2_infer_result/GA_GWAS_2022/'

model_dir = root + r'Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/'

data_dir = r'/Volumes/easystore/GWAS 2022/'
data_dir2 = r'/Volumes/easystore/'

blocks = ['Block2/','Block3/','Block8/']

## Model Inference Configuration

In [3]:
classes=['root','AMF internal hypha','AMF external hypha','AMF arbuscule','AMF vesicle','AMF spore','others']

In [4]:
cfg = get_cfg() # return default configuration
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")) # copy config files from open source projects

# training configuration
cfg.DATASETS.TEST=()
cfg.DATALOADER.NUM_WORKERS=2
#cfg.SOLVER.IMS_PER_BATCH=args.batch_size

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE=128 #Number of regions per image used to train RPN. faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES=len(classes)# (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.MODEL.BACKBONE.FREEZE_AT=2
cfg.SEED=1
cfg.AUG_FLAG=1

# inference configuration
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # set threshold for this model
cfg.MODEL.WEIGHTS=os.path.join(model_dir, "Trainset1_model_best.pth") # path to the best model trained
cfg.MODEL.DEVICE='cpu' # use cpu for inference

# I removed spore and others from inference classes
inf_metadata = MetadataCatalog.get("inference").set(thing_classes=['root','AMF internal hypha','AMF external hypha','AMF arbuscule','AMF vesicle'])

In [5]:
predictor = DefaultPredictor(cfg)

[10/02 22:14:18 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


## Prepare for parallel inference

In [8]:
allpath2block2img = [os.path.join(path,name) for path, dirs, files in os.walk(os.path.join(data_dir,'Block2/'))
                        for name in files
                        if name.endswith('.czi')]
allpath2block8img = [os.path.join(path,name) for path, dirs, files in os.walk(os.path.join(data_dir,'Block8/'))
                        for name in files
                        if name.endswith('.czi')]

In [113]:
allpath2block10img = [os.path.join(path,name) for path, dirs, files in os.walk(os.path.join(data_dir2,'Block10/'))
                        for name in files
                        if name.endswith('.czi')]

In [103]:
# Get file sizes for each file
file_sizes = [(path, os.path.getsize(path)) for path in allpath2block2img]
# Sort the file paths based on file sizes
sorted_path2block2img = [item[0] for item in sorted(file_sizes, key=lambda x: x[1], reverse=True)]

file_sizes = [(path, os.path.getsize(path)) for path in allpath2block8img]
sorted_path2block8img = [item[0] for item in sorted(file_sizes, key=lambda x: x[1], reverse=True)]



In [114]:
file_sizes = [(path, os.path.getsize(path)) for path in allpath2block10img]
sorted_path2block10img = [item[0] for item in sorted(file_sizes, key=lambda x: x[1], reverse=True)]


In [115]:
print(len(sorted_path2block2img))
print(len(sorted_path2block8img))
print(len(sorted_path2block10img))

344
384
337


### Calculate per-channel means for images in each block
Take a sample of 30 scenes to estimate per channel means

In [23]:
def findBGRmean(file):
    czi = AICSImage(file)
    scenes = czi.scenes
    x = random.choice(scenes)
    czi.set_scene(x)
    img_array = czi.get_image_data("YXS", T=0,C=0,Z=0)
    # Convert numpy ndarray to PyTorch tensor
    img_tensor = torch.from_numpy(img_array).float()  # Convert data type to float
    # Change the dimensions to (channels, height, width)
    img_tensor = img_tensor.permute(2, 0, 1)
    # Calculate the mean for each channel
    mean_values = img_tensor.mean(dim=[1, 2])
    return mean_values.unsqueeze(0)

In [42]:
@ray.remote
def rfindBGRmean(file):
    czi = AICSImage(file)
    scenes = czi.scenes
    x = random.choice(scenes)
    czi.set_scene(x)
    img_array = czi.get_image_data("YXS", T=0,C=0,Z=0)
    # Convert numpy ndarray to PyTorch tensor
    img_tensor = torch.from_numpy(img_array).float()  # Convert data type to float
    # Change the dimensions to (channels, height, width)
    img_tensor = img_tensor.permute(2, 0, 1)
    # Calculate the mean for each channel
    mean_values = img_tensor.mean(dim=[1, 2])
    return mean_values.unsqueeze(0)

#### Block 2 images

In [21]:
# randomly choose 30 scenes from block2 images
sample30 = random.sample(allpath2block2img,30)
file_sizes = [(path, os.path.getsize(path)) for path in sample30]

In [44]:
ray.init(num_cpus=18, ignore_reinit_error=True)

2023-10-02 23:50:01,149	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.18
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [45]:
id30 = [rfindBGRmean.remote(czi) for czi in sample30]
results = []
t0 = time.time()
for i in range(1,40):
    ready, not_ready = ray.wait(id30, num_returns = 1)
    print('iteration:', i) 
    results.extend(ray.get(ready))
    del ready
    id30 = not_ready
    if not id30: 
        break
print('Time Elapsed:\t{:.4f}'.format(time.time() - t0))

iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 20
iteration: 21
iteration: 22
iteration: 23
iteration: 24
iteration: 25
iteration: 26
iteration: 27
iteration: 28
iteration: 29
iteration: 30
Time Elapsed:	34.5561


In [43]:
ray.shutdown()

In [46]:
print(results)

[tensor([[174.9772, 174.4386, 173.9380]]), tensor([[135.9655, 135.5739, 135.5250]]), tensor([[162.2106, 158.3190, 157.7910]]), tensor([[149.1607, 147.1816, 146.7908]]), tensor([[170.2700, 168.7100, 168.2848]]), tensor([[152.6491, 150.6038, 150.2349]]), tensor([[116.8860, 115.9650, 114.6433]]), tensor([[132.3646, 132.6676, 132.8183]]), tensor([[126.8691, 125.2042, 124.5668]]), tensor([[143.1965, 141.5301, 141.2795]]), tensor([[124.6719, 123.9459, 123.7732]]), tensor([[149.2669, 147.0962, 146.4940]]), tensor([[126.3323, 123.8550, 123.2347]]), tensor([[137.9844, 137.6405, 137.1638]]), tensor([[148.0428, 147.6507, 146.1341]]), tensor([[124.5397, 123.0701, 122.6528]]), tensor([[128.9459, 126.8098, 126.2509]]), tensor([[104.0251, 103.3973, 103.3155]]), tensor([[119.4886, 116.3476, 115.5772]]), tensor([[102.5644,  99.8095,  99.2338]]), tensor([[107.3763, 105.3407, 104.7592]]), tensor([[128.4014, 127.4890, 127.3115]]), tensor([[162.4699, 160.4639, 159.9668]]), tensor([[142.3185, 139.3402, 138.

In [47]:
results_ts = torch.cat(results, dim=0)
block2BGRmeans = results_ts.mean(dim=0)

In [ ]:
print(block)

#### Block 8

In [50]:
sample30 = random.sample(allpath2block8img,30)
file_sizes = [(path, os.path.getsize(path)) for path in sample30]

In [51]:
ray.init(num_cpus=18,ignore_reinit_error=True)

2023-10-02 23:56:55,202	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.


Python version:,3.8.18
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [52]:
id30 = [rfindBGRmean.remote(czi) for czi in sample30]
results = []
t0 = time.time()
for i in range(1,40):
    ready, not_ready = ray.wait(id30, num_returns = 1)
    print('iteration:', i) 
    results.extend(ray.get(ready))
    del ready
    id30 = not_ready
    if not id30: 
        break
print('Time Elapsed:\t{:.4f}'.format(time.time() - t0))

iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 20
iteration: 21
iteration: 22
iteration: 23
iteration: 24
iteration: 25
iteration: 26
iteration: 27
iteration: 28
iteration: 29
iteration: 30
Time Elapsed:	100.3987


In [53]:
ray.shutdown()

In [54]:
results_ts = torch.cat(results, dim=0)
block8BGRmeans = results_ts.mean(dim=0)

#### Block 10

In [55]:
sample30 = random.sample(allpath2block10img,30)
file_sizes = [(path, os.path.getsize(path)) for path in sample30]

In [56]:
ray.init(num_cpus=18,ignore_reinit_error=True)

2023-10-02 23:59:44,015	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.18
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [57]:
id30 = [rfindBGRmean.remote(czi) for czi in sample30]
results = []
t0 = time.time()
for i in range(1,40):
    ready, not_ready = ray.wait(id30, num_returns = 1)
    print('iteration:', i) 
    results.extend(ray.get(ready))
    del ready
    id30 = not_ready
    if not id30: 
        break
print('Time Elapsed:\t{:.4f}'.format(time.time() - t0))

iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 20
iteration: 21
iteration: 22
iteration: 23
iteration: 24
iteration: 25
iteration: 26
iteration: 27
iteration: 28
iteration: 29
iteration: 30
Time Elapsed:	99.6823


In [74]:
ray.shutdown()

In [59]:
results_ts = torch.cat(results, dim=0)
block10BGRmeans = results_ts.mean(dim=0)

### Calculate per-channel difference for centering images

In [60]:
train_means = torch.tensor([145.85450147081923, 152.11404784018066, 161.0139541272665])

In [64]:
diff_blk2 = train_means - block2BGRmeans
diff_blk8 = train_means - block8BGRmeans
diff_blk10 = train_means - block10BGRmeans

## Set up Ray Remote function

In [68]:
def centering2train(diff, img, x, y):
    '''
    :param diff: 3d tensor (row, col, 3)
    :param img: np.ndarray (row, col ,3)
    :return img:
    :rtype ndnumpy.array:
    Objective: output an image centered using training set per-channel means
    '''
    
    # Convert diff to a numpy array and reshape it to (3, 1, 1)
    diff_array = np.array(diff).reshape(3, 1, 1)
    # Broadcast to shape (3, imageHeight, imgWidth)
    expanded_array = np.broadcast_to(diff_array, (3, y, x))
    # Transpose to get shape (imageHeight, imgWidth, 3)
    diff_ts = expanded_array.transpose(1, 2, 0)
    
    img = np.add(img,diff_ts)
    # Clip values outside the interval are clipped to the interval edges
    np.clip(img, 0, 255, out=img)
    return img

def padImg(img, x, y, tilex, tiley):
    '''
    :param img:
    :return padded img:
    :rtype ndnumpy.array:
    Objective: output a padded image dividle by tile size
    '''
    pad_top = tiley - (y % tiley)
    pad_lft = tilex - (x % tilex)
    img = cv2.copyMakeBorder(img,pad_top,0,pad_lft,0,cv2.BORDER_CONSTANT,value=[0,0,0])
    return img

In [75]:
@ray.remote
def inference(pathtofile, block, diff, predictor):
    blklist = []
    imgidlist = []
    sceneidlist = []
    tileidlist = []
    classlist = []
    confscorelist=[]
    arealist = []

    # read in czi
    czi = AICSImage(pathtofile)
   
    for scene in czi.scenes:
        print(scene)
        # extract image by scene
        czi.set_scene(scene)
        img = czi.get_image_data("YXS", T=0,C=0,Z=0) # numpy.ndarray  
        y = img.shape[0]
        x = img.shape[1]
        # centering
        img = centering2train(diff, img, x, y)
        # pad image
        img = padImg(img, x, y, 2560, 1920)
        # tiling
        for i in range(0,y,1920):
            for j in range(0,x,2560):
                xmin = j
                xmax = j + 2560
                ymin = i
                ymax = i + 1920
                tile_id = str(xmin)+"_"+str(ymin)+"_"+str(xmax)+"_"+str(ymax)
                subimg = img[ymin:ymax,xmin:xmax]
                outputs = predictor(subimg)
                        
                #inference outputs
                clasind = outputs['instances'].get('pred_classes')
                allmasks = outputs['instances'].get('pred_masks')
                allscores = outputs['instances'].get('scores')
            
                num_seg = clasind.size()[0]
                if num_seg != 0: # only save an entry when the image contains a segmentation
                    blklist = blklist + np.repeat(block[:-1], num_seg).tolist()
                    imgidlist = imgidlist + np.repeat(pathtofile, num_seg).tolist()
                    sceneidlist = sceneidlist + np.repeat(scene, num_seg).tolist()
                    tileidlist = tileidlist + np.repeat(tile_id, num_seg).tolist()
                    confscorelist = confscorelist + allscores.tolist()
                
                    # calculate the area of segmentation
                    v = Visualizer(subimg[:, :, ::-1], MetadataCatalog.get("inference"), scale=1.0)
                    for i in range(0,num_seg,1):
                        #calculate mask area
                        locmask = np.asarray(allmasks[i,:,:])
                        gmask = GenericMask(locmask,v.output.height,v.output.width)
                        if gmask.polygons:
                            mergpolygon = gmask.polygons[0]
                            all_points_x = mergpolygon[::2]
                            all_points_y = mergpolygon[1::2]
                            pgon = Polygon(zip(all_points_x,all_points_y))
                            arealist.append(pgon.area)
                            # class index to class name
                            classlist.append(classes[clasind.tolist()[i]])
                        else: # assign NAs to non-polygon mask
                            arealist.append(math.nan)
                            classlist.append(math.nan)

    # export inference result as df
    infresults = pd.DataFrame({
    'block': blklist,
    'filename': imgidlist,
    'scene': sceneidlist,
    'tile': tileidlist,
    'annotations': classlist,
    'area': arealist,
    'confidenceScore': confscorelist})
    
    # delete 
    return infresults

## Parallel inference on block 2

In [76]:
results = []

batch_size = 15
t0 = time.time()
for i in range(0, len(sorted_path2block2img), batch_size):
    batch = sorted_path2block2img[i:i+batch_size] # if the remaining images are less than 10, the last batch will be smaller
    
    ray.init(num_cpus=18,ignore_reinit_error=True)
    ids = [inference.remote(path, 'Block2/', diff_blk2, predictor) 
        for path in batch]
    
    t1 = time.time()
    for task in range(1,20,1):
        ntask = i + task
        print('image:', ntask)
        
        ready, not_ready = ray.wait(ids, num_returns = 1)
        results.extend(ray.get(ready))
        
        del ready #%% Clean-up object store data 
        ids = not_ready
        if not ids: 
            break
        print('Time Elapsed:\t{:.4f}'.format(time.time() - t1))
    ray.shutdown()
    print('Batch Time Elapsed:\t{:.4f}'.format(time.time() - t1))
print('Total Time Elapsed:\t{:.4f}'.format(time.time() - t0))

2023-10-03 00:42:44,258	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


image: 1
(inference pid=48042) ScanRegion0
(inference pid=48039) ScanRegion1 [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(inference pid=48037) ScanRegion1
(inference pid=48041) ScanRegion1
(inference pid=48042) ScanRegion1
(inference pid=48043) ScanRegion1
(inference pid=48038) ScanRegion1
(inference pid=48036) ScanRegion1
(inference pid=48042) ScanRegion2 [repeated 2x across cluster]
(inference pid=48038) ScanRegion2
(inference pid=48037) ScanRegion2
(inference pid=48039) ScanRegion2
(inference pid=48041) ScanRegion2
(inference pid=48042) ScanRegion3 [repeated 4x across cluster]
(inference pid=48039) ScanRegion3
(inference pid=48041) ScanRegion3
(inference pid=48042) ScanRegion4 [repeated 4x across cluster]
(inference pid=48043) ScanRegion4
(inference pid=48031) ScanRegion3
(inference pid=48036) ScanRegion4 

(inference pid=48041) ScanRegion22
(inference pid=48039) ScanRegion2
(inference pid=48038) ScanRegion23
(inference pid=48037) ScanRegion23
Time Elapsed:	1662.0663
image: 4
(inference pid=48031) ScanRegion0
Time Elapsed:	1668.8873
image: 5
(inference pid=48043) ScanRegion0
Time Elapsed:	1682.7767
image: 6
(inference pid=48036) ScanRegion0 [repeated 2x across cluster]
(inference pid=48041) ScanRegion23 [repeated 2x across cluster]
Time Elapsed:	1714.2054
image: 7
(inference pid=48037) ScanRegion1
(inference pid=48043) ScanRegion1
(inference pid=48031) ScanRegion1
Time Elapsed:	1753.5668
image: 8
(inference pid=48038) ScanRegion0
(inference pid=48037) ScanRegion2 [repeated 2x across cluster]
(inference pid=48031) ScanRegion2 [repeated 2x across cluster]
(inference pid=48041) ScanRegion24
(inference pid=48036) ScanRegion1
(inference pid=48031) ScanRegion3 [repeated 2x across cluster]
(inference pid=48038) ScanRegion1 [repeated 2x across cluster]
(inference pid=48042) ScanRegion8
Time Elaps

2023-10-03 01:40:26,623	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


image: 16
(inference pid=48566) ScanRegion0
(inference pid=48568) ScanRegion1 [repeated 8x across cluster]
(inference pid=48572) ScanRegion1 [repeated 2x across cluster]
(inference pid=48567) ScanRegion1 [repeated 2x across cluster]
(inference pid=48568) ScanRegion2
(inference pid=48571) ScanRegion1
(inference pid=48564) ScanRegion1
(inference pid=48567) ScanRegion2
(inference pid=48565) ScanRegion1
(inference pid=48566) ScanRegion2
(inference pid=48572) ScanRegion2 [repeated 2x across cluster]
(inference pid=48567) ScanRegion3
(inference pid=48568) ScanRegion3
(inference pid=48569) ScanRegion3
(inference pid=48571) ScanRegion2
(inference pid=48572) ScanRegion3
(inference pid=48565) ScanRegion2
(inference pid=48564) ScanRegion2 [repeated 2x across cluster]
(inference pid=48571) ScanRegion3
(inference pid=48567) ScanRegion4
(inference pid=48568) ScanRegion4
(inference pid=48572) ScanRegion4
(inference pid=48569) ScanRegion4 [repeated 2x across cluster]
(inference pid=48566) ScanRegion5 

...

(inference pid=65138) ScanRegion16
(inference pid=65136) ScanRegion28 [repeated 2x across cluster]
(inference pid=65142) ScanRegion19
(inference pid=65131) ScanRegion17
(inference pid=65141) ScanRegion29
(inference pid=65140) ScanRegion27
(inference pid=65136) ScanRegion29 [repeated 2x across cluster]
(inference pid=65141) ScanRegion30
(inference pid=65138) ScanRegion17
(inference pid=65143) ScanRegion21
(inference pid=65136) ScanRegion30
(inference pid=65142) ScanRegion20 [repeated 2x across cluster]
(inference pid=65131) ScanRegion18
(inference pid=65141) ScanRegion31
(inference pid=65136) ScanRegion31
(inference pid=65140) ScanRegion29
(inference pid=65143) ScanRegion22
(inference pid=65138) ScanRegion18
(inference pid=65136) ScanRegion32
(inference pid=65141) ScanRegion32
(inference pid=65142) ScanRegion21
(inference pid=65131) ScanRegion19
(inference pid=65140) ScanRegion30
(inference pid=65136) ScanRegion33
(inference pid=65140) ScanRegion31 [repeated 2x across cluster]
(inferenc

2023-10-04 08:59:11,467	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


image: 331
(inference pid=66145) ScanRegion0
(inference pid=66136) ScanRegion0 [repeated 7x across cluster]
(inference pid=66136) ScanRegion1
(inference pid=66145) ScanRegion1
(inference pid=66137) ScanRegion1 [repeated 2x across cluster]
(inference pid=66147) ScanRegion1
(inference pid=66143) ScanRegion1
(inference pid=66144) ScanRegion1
(inference pid=66146) ScanRegion1
(inference pid=66137) ScanRegion2 [repeated 2x across cluster]
(inference pid=66136) ScanRegion2
(inference pid=66134) ScanRegion2
(inference pid=66147) ScanRegion2 [repeated 2x across cluster]
(inference pid=66144) ScanRegion2 [repeated 3x across cluster]
(inference pid=66146) ScanRegion2
(inference pid=66145) ScanRegion4
(inference pid=66144) ScanRegion3
(inference pid=66134) ScanRegion3
(inference pid=66147) ScanRegion3
(inference pid=66136) ScanRegion4
(inference pid=66143) ScanRegion3
(inference pid=66137) ScanRegion3
(inference pid=66145) ScanRegion5
(inference pid=66144) ScanRegion4
(inference pid=66146) ScanRe

(inference pid=66137) ScanRegion21
(inference pid=66147) ScanRegion24
(inference pid=66134) ScanRegion24
(inference pid=66146) ScanRegion16 [repeated 2x across cluster]
Time Elapsed:	3640.7944
image: 332
(inference pid=66144) ScanRegion26
(inference pid=66145) ScanRegion0
(inference pid=66136) ScanRegion29
(inference pid=66134) ScanRegion25
(inference pid=66143) ScanRegion24
Time Elapsed:	3730.3597
image: 333
(inference pid=66147) ScanRegion0
(inference pid=66137) ScanRegion22
Time Elapsed:	3754.6695
image: 334
(inference pid=66136) ScanRegion0
(inference pid=66145) ScanRegion1
(inference pid=66147) ScanRegion1
(inference pid=66134) ScanRegion26
(inference pid=66144) ScanRegion27
(inference pid=66145) ScanRegion2
(inference pid=66136) ScanRegion1
Time Elapsed:	3859.2956
image: 335
(inference pid=66143) ScanRegion0
(inference pid=66136) ScanRegion2
(inference pid=66147) ScanRegion2
(inference pid=66145) ScanRegion3
Time Elapsed:	3972.4169
image: 336
(inference pid=66137) ScanRegion24 [r

(inference pid=66143) ScanRegion26
(inference pid=66143) ScanRegion27
(inference pid=66143) ScanRegion28
Time Elapsed:	7246.9326
image: 343
(inference pid=66144) ScanRegion16 [repeated 2x across cluster]
(inference pid=66134) ScanRegion17
(inference pid=66144) ScanRegion17
(inference pid=66134) ScanRegion18
(inference pid=66134) ScanRegion19
(inference pid=66144) ScanRegion18
(inference pid=66144) ScanRegion19
(inference pid=66134) ScanRegion20
(inference pid=66144) ScanRegion20
(inference pid=66144) ScanRegion21
(inference pid=66144) ScanRegion22
(inference pid=66144) ScanRegion23
(inference pid=66144) ScanRegion24
(inference pid=66134) ScanRegion21
(inference pid=66144) ScanRegion25
(inference pid=66144) ScanRegion26
(inference pid=66134) ScanRegion22
(inference pid=66144) ScanRegion27
(inference pid=66144) ScanRegion28
(inference pid=66144) ScanRegion29
(inference pid=66144) ScanRegion30
(inference pid=66144) ScanRegion31
(inference pid=66134) ScanRegion23
Time Elapsed:	8082.4101
im

In [77]:
# save result as dataframe
df_blk2 = pd.concat(results, axis=0)

In [81]:
# quick quality control
print(df_blk2.shape)
print(len(df_blk2['filename'].unique()))

(56456, 7)
344


In [82]:
# export result as csv file 
df_blk2.to_csv(os.path.join(output_dir,"block2_segmentation_image_complete.txt"),index=False)

## Debug: only 8 cpus are used for parallel inference
There were only 8 unique pids in the output above, which indicated that 8 out of 18 asked cpus were allocated for parallel inference. This imac only has 10 physical cores in the cpu but with hyper-threading each core can execute 2 thread concurrently. Therefore, the number of logical processors is 20, twice the number of cores. 

A logical core (i.e. HyperThreading) works by swapping in a thread of execution onto a physical core when the current thread on the physical core stalls to e.g. access memory or perform I/O. If all threads involve mostly on-physical-core execution and therefore are performing very little memory accesses or I/O, these threads will end up being in constant contention for the physical core, yielding very little parallelism between the threads, and maybe even degrading performance due to the overhead of these context switches.

Maybe the kind of tasks I am performing are not suitable for hyperthreading. Also, I will try to parallelize by scene. This will reduce the amount of memroy used for reading and writing files. 

## Parallel tasks by scene

In [99]:
@ray.remote
def inferenceByScene(czi_ref, imgname, scene, block, diff, predictor):
    blklist = []
    imgidlist = []
    sceneidlist = []
    tileidlist = []
    classlist = []
    confscorelist=[]
    arealist = []


    # extract image by scene
    czi_ref.set_scene(scene)
    print(czi_ref.current_scene)
    img = czi_ref.get_image_data("YXS", T=0,C=0,Z=0) # numpy.ndarray  
    y = img.shape[0]
    x = img.shape[1]
        
    # centering
    img = centering2train(diff, img, x, y)
    # pad image
    img = padImg(img, x, y, 2560, 1920)
    # tiling
    for i in range(0,y,1920):
        for j in range(0,x,2560):
            xmin = j
            xmax = j + 2560
            ymin = i
            ymax = i + 1920
            tile_id = str(xmin)+"_"+str(ymin)+"_"+str(xmax)+"_"+str(ymax)
            subimg = img[ymin:ymax,xmin:xmax]
            outputs = predictor(subimg)
                        
            #inference outputs
            clasind = outputs['instances'].get('pred_classes')
            allmasks = outputs['instances'].get('pred_masks')
            allscores = outputs['instances'].get('scores')
            
            num_seg = clasind.size()[0]
            if num_seg != 0: # only save an entry when the image contains a segmentation
                blklist = blklist + np.repeat(block[:-1], num_seg).tolist()
                imgidlist = imgidlist + np.repeat(imgname, num_seg).tolist()
                sceneidlist = sceneidlist + np.repeat(scene, num_seg).tolist()
                tileidlist = tileidlist + np.repeat(tile_id, num_seg).tolist()
                confscorelist = confscorelist + allscores.tolist()
                
                # calculate the area of segmentation
                v = Visualizer(subimg[:, :, ::-1], MetadataCatalog.get("inference"), scale=1.0)
                for i in range(0,num_seg,1):
                    #calculate mask area
                    locmask = np.asarray(allmasks[i,:,:])
                    gmask = GenericMask(locmask,v.output.height,v.output.width)
                    if gmask.polygons:
                        mergpolygon = gmask.polygons[0]
                        all_points_x = mergpolygon[::2]
                        all_points_y = mergpolygon[1::2]
                        pgon = Polygon(zip(all_points_x,all_points_y))
                        arealist.append(pgon.area)
                        # class index to class name
                        classlist.append(classes[clasind.tolist()[i]])
                    else: # assign NAs to non-polygon mask
                        arealist.append(math.nan)
                        classlist.append(math.nan)

    # export inference result as df
    infresults = pd.DataFrame({
    'block': blklist,
    'filename': imgidlist,
    'scene': sceneidlist,
    'tile': tileidlist,
    'annotations': classlist,
    'area': arealist,
    'confidenceScore': confscorelist})
    
    return infresults



1. put 1 image in Ray's shared memory with ray.put()
2. paralleize tasks by scene in the image in shared memory
3. do inference on 1 block2 images and see how many cpus are being used

In [101]:
testresults = []
t0 = time.time()

ray.init(num_cpus=18,ignore_reinit_error=True)
t1 = time.time()
czifile = ray.put(AICSImage(allpath2block2img[0])) 
scenes = ray.get(czifile).scenes
print(scenes)
ids = [inferenceByScene.remote(czifile, allpath2block2img[0], scene, 'Block2/', diff_blk2, predictor) 
       for scene in scenes]
print('Init Time Elapse:\t{:.4f}'.format(time.time() - t1))
t2 = time.time()
for j in range(0, len(scenes),1):
    ready, not_ready = ray.wait(ids, num_returns = 1)
    print('Scene:', j+1)
    testresults.extend(ray.get(ready))
    del ready #%% Clean-up object store data 
    ids = not_ready
    if not ids: 
        break # this will only break the inner loop j
    print('Scene Time Elapsed:\t{:.4f}'.format(time.time() - t2))
del czifile
print('Image Time Elapsed:\t{:.4f}'.format(time.time() - t2))

ray.shutdown()

2023-10-06 00:01:03,094	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


('ScanRegion0', 'ScanRegion1', 'ScanRegion2', 'ScanRegion3', 'ScanRegion4', 'ScanRegion5', 'ScanRegion6', 'ScanRegion7', 'ScanRegion8', 'ScanRegion9', 'ScanRegion10', 'ScanRegion11', 'ScanRegion12', 'ScanRegion13', 'ScanRegion14', 'ScanRegion15', 'ScanRegion16', 'ScanRegion17', 'ScanRegion18', 'ScanRegion19', 'ScanRegion20', 'ScanRegion21', 'ScanRegion22')
(inferenceByScene pid=86858) ScanRegion0
Init Time Elapse:	11.4174


(raylet) Spilled 2354 MiB, 14 objects, write throughput 639 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


Scene: 1
Scene Time Elapsed:	76.8952
(inferenceByScene pid=86855) ScanRegion8 [repeated 8x across cluster]
Scene: 2
Scene Time Elapsed:	99.8922
Scene: 3
Scene Time Elapsed:	100.5020
(inferenceByScene pid=86858) ScanRegion9
(inferenceByScene pid=86867) ScanRegion10
Scene: 4
Scene Time Elapsed:	103.1435
Scene: 5
Scene Time Elapsed:	104.3290
Scene: 6
Scene Time Elapsed:	122.1827
(inferenceByScene pid=86868) ScanRegion13 [repeated 3x across cluster]
Scene: 7
Scene Time Elapsed:	125.5897
Scene: 8
Scene Time Elapsed:	153.2416
(inferenceByScene pid=86870) ScanRegion15 [repeated 2x across cluster]
Scene: 9
Scene Time Elapsed:	203.2996
(inferenceByScene pid=86855) ScanRegion16
Scene: 10
Scene Time Elapsed:	216.8849
(inferenceByScene pid=86868) ScanRegion17
Scene: 11
Scene Time Elapsed:	225.8686
(inferenceByScene pid=86858) ScanRegion18
Scene: 12
Scene Time Elapsed:	226.7926
(inferenceByScene pid=86867) ScanRegion19
Scene: 13
Scene Time Elapsed:	235.2373
(inferenceByScene pid=86856) ScanRegion20

In [102]:
testresults = []
t0 = time.time()

ray.init(num_cpus=8,ignore_reinit_error=True)
t1 = time.time()
czifile = ray.put(AICSImage(allpath2block2img[0])) 
scenes = ray.get(czifile).scenes
print(scenes)
ids = [inferenceByScene.remote(czifile, allpath2block2img[0], scene, 'Block2/', diff_blk2, predictor) 
       for scene in scenes]
print('Init Time Elapse:\t{:.4f}'.format(time.time() - t1))
t2 = time.time()
for j in range(0, len(scenes),1):
    ready, not_ready = ray.wait(ids, num_returns = 1)
    print('Scene:', j+1)
    testresults.extend(ray.get(ready))
    del ready #%% Clean-up object store data 
    ids = not_ready
    if not ids: 
        break # this will only break the inner loop j
    print('Scene Time Elapsed:\t{:.4f}'.format(time.time() - t2))
del czifile
print('Image Time Elapsed:\t{:.4f}'.format(time.time() - t2))

ray.shutdown()

2023-10-06 00:09:24,007	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


('ScanRegion0', 'ScanRegion1', 'ScanRegion2', 'ScanRegion3', 'ScanRegion4', 'ScanRegion5', 'ScanRegion6', 'ScanRegion7', 'ScanRegion8', 'ScanRegion9', 'ScanRegion10', 'ScanRegion11', 'ScanRegion12', 'ScanRegion13', 'ScanRegion14', 'ScanRegion15', 'ScanRegion16', 'ScanRegion17', 'ScanRegion18', 'ScanRegion19', 'ScanRegion20', 'ScanRegion21', 'ScanRegion22')
(inferenceByScene pid=87020) ScanRegion0


(raylet) Spilled 2186 MiB, 13 objects, write throughput 585 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


Init Time Elapse:	12.3323
Scene: 1
Scene Time Elapsed:	72.2484
(inferenceByScene pid=87018) ScanRegion8 [repeated 8x across cluster]
Scene: 2
Scene Time Elapsed:	95.4218
(inferenceByScene pid=87020) ScanRegion9
Scene: 3
Scene Time Elapsed:	95.9204
(inferenceByScene pid=87021) ScanRegion10
Scene: 4
Scene Time Elapsed:	99.1977
Scene: 5
Scene Time Elapsed:	100.4917
Scene: 6
Scene Time Elapsed:	117.3868
(inferenceByScene pid=87019) ScanRegion13 [repeated 3x across cluster]
Scene: 7
Scene Time Elapsed:	119.8852
Scene: 8
Scene Time Elapsed:	148.6063
(inferenceByScene pid=87014) ScanRegion15 [repeated 2x across cluster]
Scene: 9
Scene Time Elapsed:	196.8253
(inferenceByScene pid=87018) ScanRegion16
Scene: 10
Scene Time Elapsed:	211.3072
(inferenceByScene pid=87019) ScanRegion17
Scene: 11
Scene Time Elapsed:	219.9219
(inferenceByScene pid=87020) ScanRegion18
Scene: 12
Scene Time Elapsed:	221.7444
(inferenceByScene pid=87021) ScanRegion19
Scene: 13
Scene Time Elapsed:	230.3065
(inferenceByScene

## Parallel inference on block 8

In [106]:
results = []
t0 = time.time()

ray.init(num_cpus=8,ignore_reinit_error=True)
for i in range(0, 400, 1):
    t1 = time.time()
    czifile = ray.put(AICSImage(sorted_path2block8img[i])) 
    scenes = ray.get(czifile).scenes
    ids = [inferenceByScene.remote(czifile, sorted_path2block8img[i], scene, 'Block8/', diff_blk8, predictor) 
        for scene in scenes]
    print('Init Time Elapse:\t{:.4f}'.format(time.time() - t1))
    t2 = time.time()
    print('Image:',i+1)
    for j in range(0, len(scenes),1):
        ready, not_ready = ray.wait(ids, num_returns = 1)
        results.extend(ray.get(ready))
        del ready #%% Clean-up object store data 
        ids = not_ready
        if not ids: 
            break # this will only break the inner loop j
        print('Scene Time Elapsed:\t{:.4f}'.format(time.time() - t2))
    del czifile
    print('Image Time Elapsed:\t{:.4f}'.format(time.time() - t2))
print('Total Time Elapsed:\t{:.4f}'.format(time.time() - t0))
ray.shutdown()

2023-10-06 00:24:21,805	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(inferenceByScene pid=87175) ScanRegion0


(raylet) Spilled 2354 MiB, 14 objects, write throughput 733 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 4205 MiB, 25 objects, write throughput 736 MiB/s.


Init Time Elapse:	20.1197
Image: 1
Scene Time Elapsed:	83.4366
(inferenceByScene pid=87179) ScanRegion8 [repeated 8x across cluster]
Scene Time Elapsed:	138.8426
(inferenceByScene pid=87177) ScanRegion9
Scene Time Elapsed:	173.1732
(inferenceByScene pid=87175) ScanRegion10
Scene Time Elapsed:	177.7523
Scene Time Elapsed:	179.1086
(inferenceByScene pid=87176) ScanRegion12 [repeated 2x across cluster]
Scene Time Elapsed:	180.7487
Scene Time Elapsed:	181.8781
Scene Time Elapsed:	224.3464
(inferenceByScene pid=87181) ScanRegion15 [repeated 3x across cluster]
Scene Time Elapsed:	266.9648
(inferenceByScene pid=87175) ScanRegion16
Scene Time Elapsed:	271.2520
(inferenceByScene pid=87179) ScanRegion17
Scene Time Elapsed:	273.3868
Scene Time Elapsed:	300.7472
(inferenceByScene pid=87178) ScanRegion19 [repeated 2x across cluster]
Scene Time Elapsed:	315.9090
(inferenceByScene pid=87176) ScanRegion20
Scene Time Elapsed:	324.2163
Scene Time Elapsed:	324.3848
(inferenceByScene pid=87177) ScanRegion

(raylet) Spilled 8241 MiB, 49 objects, write throughput 731 MiB/s.


...

(inferenceByScene pid=87175) CZI file appears to contain multiple scenes but dimension data is not available in this file. Root node of split multi-scene CZI files are not supported by CziReader.


RayTaskError(PylibCZI_CDimCoordinatesOverspecifiedException): [36mray::inferenceByScene()[39m (pid=87175, ip=127.0.0.1)
  File "/var/folders/mv/p4jr8xs52gl7rk_wf4pn6mn80000gn/T/ipykernel_46033/3254502686.py", line 15, in inferenceByScene
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/aicsimageio/aics_image.py", line 728, in get_image_data
    data=self.data,
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/aicsimageio/aics_image.py", line 542, in data
    return self.xarray_data.data
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/aicsimageio/aics_image.py", line 501, in xarray_data
    self.reader.xarray_data
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/aicsimageio/readers/reader.py", line 372, in xarray_data
    self._xarray_data = self._read_immediate()
  File "/Users/lovely_shufan/opt/anaconda3/envs/dete

(inferenceByScene pid=87175) CZI file appears to contain multiple scenes but dimension data is not available in this file. Root node of split multi-scene CZI files are not supported by CziReader.


(inferenceByScene pid=87175) ScanRegion27


2023-10-07 21:47:23,080	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::inferenceByScene() (pid=87175, ip=127.0.0.1)
  File "/var/folders/mv/p4jr8xs52gl7rk_wf4pn6mn80000gn/T/ipykernel_46033/3254502686.py", line 15, in inferenceByScene
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/aicsimageio/aics_image.py", line 728, in get_image_data
    data=self.data,
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/aicsimageio/aics_image.py", line 542, in data
    return self.xarray_data.data
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/aicsimageio/aics_image.py", line 501, in xarray_data
    self.reader.xarray_data
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/aicsimageio/readers/reader.py", line 372, in xarray_data
    self._xarray_data = self._read_immediate()
  File "/Use

In [107]:
# save result as dataframe
df_blk8 = pd.concat(results, axis=0)

# quick quality control
print(df_blk8.shape)
print(len(df_blk8['filename'].unique()))

(65339, 7)
382


In [108]:
# export result as csv file 
df_blk8.to_csv(os.path.join(output_dir,"block8_segmentation_image_1to382.txt"),index=False)

In [118]:
print(len(sorted_path2block8img))

384


## Error code on image 383-384 in Block 8
Inference stopped on Image 383 with the following error message: CZI file appears to contain multiple scenes but dimension data is not available in this file. Root node of split multi-scene CZI files are not supported by CziReader.
1. find Image 383 in directory 

In [122]:
print(sorted_path2block8img[382])
print(sorted_path2block8img[383])

/Volumes/easystore/GWAS 2022/Block8/PI585452/PI585452_C08_R07_FM-2.czi
/Volumes/easystore/GWAS 2022/Block8/PI329471/PI329471_C01_R07_FM.czi


In [124]:
img383 = AICSImage('/Volumes/easystore 1/GWAS 2022/Block8/PI585452/PI585452_C08_R07_FM-2.czi')

In [125]:
print(img383.dims)

<Dimensions [T: 1, C: 1, Z: 1, Y: 0, X: 0]>


The image 383 and 394 are empty and could be removed. 

## Parallel inference on Block 10

In [116]:
results = []
t0 = time.time()

ray.init(num_cpus=16,ignore_reinit_error=True)
for i in range(0, 400, 1):
    t1 = time.time()
    czifile = ray.put(AICSImage(sorted_path2block10img[i])) 
    scenes = ray.get(czifile).scenes
    ids = [inferenceByScene.remote(czifile, sorted_path2block10img[i], scene, 'Block10/', diff_blk10, predictor) 
        for scene in scenes]
    print('Init Time Elapse:\t{:.4f}'.format(time.time() - t1))
    t2 = time.time()
    print('Image:',i+1)
    for j in range(0, len(scenes),1):
        ready, not_ready = ray.wait(ids, num_returns = 1)
        results.extend(ray.get(ready))
        del ready #%% Clean-up object store data 
        ids = not_ready
        if not ids: 
            break # this will only break the inner loop j
        print('Scene Time Elapsed:\t{:.4f}'.format(time.time() - t2))
    del czifile
    print('Image Time Elapsed:\t{:.4f}'.format(time.time() - t2))
print('Total Time Elapsed:\t{:.4f}'.format(time.time() - t0))
ray.shutdown()

2023-10-08 00:50:30,315	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.


(inferenceByScene pid=87175) ScanRegion0
Init Time Elapse:	20.4598
Image: 1
Scene Time Elapsed:	153.2494
(inferenceByScene pid=87178) ScanRegion8 [repeated 8x across cluster]
Scene Time Elapsed:	175.5821
(inferenceByScene pid=87182) ScanRegion9
Scene Time Elapsed:	177.4083
(inferenceByScene pid=87176) ScanRegion10
Scene Time Elapsed:	179.8117
Scene Time Elapsed:	181.1787
Scene Time Elapsed:	211.1651
(inferenceByScene pid=87175) ScanRegion13 [repeated 3x across cluster]
Scene Time Elapsed:	333.6204
(inferenceByScene pid=87181) ScanRegion14
Scene Time Elapsed:	339.6857
(inferenceByScene pid=87178) ScanRegion15
Scene Time Elapsed:	368.7313
(inferenceByScene pid=87180) ScanRegion16
Scene Time Elapsed:	395.3908
(inferenceByScene pid=87182) ScanRegion17
Scene Time Elapsed:	400.6075
(inferenceByScene pid=87175) ScanRegion18
Scene Time Elapsed:	476.2316
(inferenceByScene pid=87176) ScanRegion19
Scene Time Elapsed:	477.0463
Scene Time Elapsed:	489.9118
(inferenceByScene pid=87179) ScanRegion21 

Scene Time Elapsed:	236.4126
(inferenceByScene pid=87175) ScanRegion22
Scene Time Elapsed:	253.6360
(inferenceByScene pid=87178) ScanRegion23
Scene Time Elapsed:	256.8011
(inferenceByScene pid=87176) ScanRegion24
Scene Time Elapsed:	275.3418
(inferenceByScene pid=87182) ScanRegion25
Scene Time Elapsed:	287.1254
(inferenceByScene pid=87180) ScanRegion26
Scene Time Elapsed:	299.0511
(inferenceByScene pid=87179) ScanRegion27
Scene Time Elapsed:	330.3448
Scene Time Elapsed:	330.9597
(inferenceByScene pid=87181) ScanRegion28
Scene Time Elapsed:	347.8747
(inferenceByScene pid=87177) ScanRegion30 [repeated 2x across cluster]
Scene Time Elapsed:	379.0207
(inferenceByScene pid=87178) ScanRegion31
Scene Time Elapsed:	383.3355
(inferenceByScene pid=87180) ScanRegion32
Scene Time Elapsed:	400.3932
(inferenceByScene pid=87182) ScanRegion33
Scene Time Elapsed:	402.1835
(inferenceByScene pid=87176) ScanRegion34
Scene Time Elapsed:	412.8782
(inferenceByScene pid=87175) ScanRegion35
Scene Time Elapsed:

(inferenceByScene pid=87178) ScanRegion0
Init Time Elapse:	22.5467
Image: 9
Scene Time Elapsed:	31.7999
(inferenceByScene pid=87179) ScanRegion8 [repeated 8x across cluster]
Scene Time Elapsed:	46.3808
(inferenceByScene pid=87178) ScanRegion9
Scene Time Elapsed:	66.0328
(inferenceByScene pid=87177) ScanRegion10
Scene Time Elapsed:	69.3910
Scene Time Elapsed:	70.3387
Scene Time Elapsed:	109.8178
(inferenceByScene pid=87176) ScanRegion13 [repeated 3x across cluster]
Scene Time Elapsed:	113.3506
Scene Time Elapsed:	142.9114
(inferenceByScene pid=87182) ScanRegion15 [repeated 2x across cluster]
Scene Time Elapsed:	150.4919
(inferenceByScene pid=87178) ScanRegion16
Scene Time Elapsed:	158.5061
(inferenceByScene pid=87179) ScanRegion17
Scene Time Elapsed:	174.2563
(inferenceByScene pid=87181) ScanRegion18
Scene Time Elapsed:	190.5953
(inferenceByScene pid=87177) ScanRegion19
Scene Time Elapsed:	192.8216
Scene Time Elapsed:	227.1275
(inferenceByScene pid=87180) ScanRegion21 [repeated 2x acros

Scene Time Elapsed:	237.1652
(inferenceByScene pid=87179) ScanRegion25
Scene Time Elapsed:	251.8514
(inferenceByScene pid=87177) ScanRegion26
Scene Time Elapsed:	267.0172
(inferenceByScene pid=87182) ScanRegion27
Scene Time Elapsed:	269.8783
(inferenceByScene pid=87181) ScanRegion28
Scene Time Elapsed:	280.6649
(inferenceByScene pid=87175) ScanRegion29
Scene Time Elapsed:	292.3209
(inferenceByScene pid=87179) ScanRegion30
Scene Time Elapsed:	295.2380
Scene Time Elapsed:	323.0673
(inferenceByScene pid=87180) ScanRegion32 [repeated 2x across cluster]
Scene Time Elapsed:	333.6209
(inferenceByScene pid=87176) ScanRegion33
Scene Time Elapsed:	346.8333
(inferenceByScene pid=87177) ScanRegion34
Scene Time Elapsed:	350.2137
Scene Time Elapsed:	361.9694
Scene Time Elapsed:	398.2235
Scene Time Elapsed:	408.2192
Scene Time Elapsed:	410.2649
Scene Time Elapsed:	419.3038
Scene Time Elapsed:	431.5816
Image Time Elapsed:	437.3888
(inferenceByScene pid=87176) ScanRegion0
(inferenceByScene pid=87177) S

...

Image Time Elapsed:	339.9304
(inferenceByScene pid=87182) ScanRegion0
(inferenceByScene pid=87175) ScanRegion1
Init Time Elapse:	16.4749
Image: 320
Scene Time Elapsed:	65.9382
(inferenceByScene pid=87175) ScanRegion8 [repeated 7x across cluster]
Scene Time Elapsed:	68.0995
Scene Time Elapsed:	69.4882
Scene Time Elapsed:	70.3100
Scene Time Elapsed:	82.0104
(inferenceByScene pid=87182) ScanRegion12 [repeated 4x across cluster]
Scene Time Elapsed:	93.3187
(inferenceByScene pid=87178) ScanRegion13
Scene Time Elapsed:	95.5433
(inferenceByScene pid=87177) ScanRegion14
Scene Time Elapsed:	144.8283
(inferenceByScene pid=87176) ScanRegion15
Scene Time Elapsed:	146.4251
(inferenceByScene pid=87182) ScanRegion16
Scene Time Elapsed:	153.8171
(inferenceByScene pid=87181) ScanRegion17
Scene Time Elapsed:	172.7843
(inferenceByScene pid=87179) ScanRegion18
Scene Time Elapsed:	191.5494
(inferenceByScene pid=87175) ScanRegion19
Scene Time Elapsed:	198.1417
(inferenceByScene pid=87178) ScanRegion21
Scene

Scene Time Elapsed:	95.7478
Scene Time Elapsed:	96.1785
Scene Time Elapsed:	145.1492
(inferenceByScene pid=87175) ScanRegion16 [repeated 4x across cluster]
Scene Time Elapsed:	157.2503
(inferenceByScene pid=87179) ScanRegion17
Scene Time Elapsed:	175.6021
(inferenceByScene pid=87176) ScanRegion18
Scene Time Elapsed:	176.5041
Scene Time Elapsed:	187.1990
(inferenceByScene pid=87182) ScanRegion20 [repeated 2x across cluster]
Scene Time Elapsed:	196.0219
(inferenceByScene pid=87181) ScanRegion21
Scene Time Elapsed:	197.8534
(inferenceByScene pid=87177) ScanRegion22
Scene Time Elapsed:	198.8045
Scene Time Elapsed:	247.3649
Scene Time Elapsed:	252.5251
Scene Time Elapsed:	253.3616
Scene Time Elapsed:	266.0534
Scene Time Elapsed:	271.2814
Scene Time Elapsed:	273.1746
Scene Time Elapsed:	295.9769
Image Time Elapsed:	301.5580
(inferenceByScene pid=87181) ScanRegion0 [repeated 2x across cluster]
Init Time Elapse:	16.8278
Image: 326
Scene Time Elapsed:	38.0716
(inferenceByScene pid=87175) ScanRe

(inferenceByScene pid=87182) ScanRegion16
Scene Time Elapsed:	121.2947
Scene Time Elapsed:	136.8975
(inferenceByScene pid=87178) ScanRegion18 [repeated 2x across cluster]
Scene Time Elapsed:	139.8850
Scene Time Elapsed:	144.2841
(inferenceByScene pid=87177) ScanRegion20 [repeated 2x across cluster]
Scene Time Elapsed:	170.1178
(inferenceByScene pid=87175) ScanRegion21
Scene Time Elapsed:	171.3737
Scene Time Elapsed:	171.6992
(inferenceByScene pid=87181) ScanRegion22
Scene Time Elapsed:	201.5975
(inferenceByScene pid=87178) ScanRegion24 [repeated 2x across cluster]
Scene Time Elapsed:	220.7278
(inferenceByScene pid=87180) ScanRegion25
Scene Time Elapsed:	221.1748
(inferenceByScene pid=87181) ScanRegion26
Scene Time Elapsed:	223.2405
Scene Time Elapsed:	234.2673
Scene Time Elapsed:	242.6619
Scene Time Elapsed:	251.7429
Scene Time Elapsed:	263.2386
Scene Time Elapsed:	273.8135
Scene Time Elapsed:	281.2623
Image Time Elapsed:	284.5977
(inferenceByScene pid=87180) ScanRegion0
(inferenceBySc

Scene Time Elapsed:	70.9721
(inferenceByScene pid=87179) ScanRegion13
Scene Time Elapsed:	82.7773
(inferenceByScene pid=87176) ScanRegion14
Scene Time Elapsed:	85.7866
Scene Time Elapsed:	93.6125
(inferenceByScene pid=87181) ScanRegion16 [repeated 2x across cluster]
Scene Time Elapsed:	94.9623
Scene Time Elapsed:	98.9538
(inferenceByScene pid=87182) ScanRegion18 [repeated 2x across cluster]
Scene Time Elapsed:	102.1565
Scene Time Elapsed:	125.1390
(inferenceByScene pid=87175) ScanRegion20 [repeated 2x across cluster]
Scene Time Elapsed:	135.4315
(inferenceByScene pid=87179) ScanRegion21
Scene Time Elapsed:	142.3964
(inferenceByScene pid=87181) ScanRegion22
Scene Time Elapsed:	163.4830
(inferenceByScene pid=87182) ScanRegion23
Scene Time Elapsed:	166.2402
(inferenceByScene pid=87177) ScanRegion24
Scene Time Elapsed:	171.7891
Scene Time Elapsed:	183.9335
Scene Time Elapsed:	188.0170
Scene Time Elapsed:	188.9862
Scene Time Elapsed:	195.9683
Scene Time Elapsed:	204.6045
Scene Time Elapsed:

IndexError: list index out of range

In [127]:
# save result as dataframe
df_blk10 = pd.concat(results, axis=0)

# quick quality control
print(df_blk10.shape)
print(len(df_blk10['filename'].unique()))
print(len(sorted_path2block10img))

(62796, 7)
337
337


In [128]:
# export result as csv file 
df_blk10.to_csv(os.path.join(output_dir,"block10_segmentation_image_complete.txt"),index=False)